In [2]:
import pandas as pd

# Load the data from a CSV file
input_prompt = input("Enter the raw file path: ")
df = pd.read_csv(input_prompt, parse_dates=['DateOfService'])

# Typecast date-time
df['DateOfService'] = df['DateOfService'].dt.date
df['DateTimeTo'] = pd.to_datetime(df['DateTimeTo'])
df['DateTimeFrom'] = pd.to_datetime(df['DateTimeFrom'])


# New column with FullName for exporting
df['ProviderFullName'] = df['ProviderFirstName'] + ' ' + df['ProviderLastName']

# Make new column for Drive Time in hours
df['DriveTimeHours'] = df['DriveTimeMinutes']/60

# Get list of provider unique ids
unique_id = df['ProviderId'].unique()

# Group by Provider and Date & Calculate Total daily work time and Total daily drive time per day
daily_hours = df.groupby(['ProviderId', 'ProviderFullName','DateOfService']).agg({'TimeWorkedInHours':'sum','DriveTimeHours':'sum'}).reset_index()

# Add new col with the daily total for both Work and Drive time
daily_hours = daily_hours.assign(TotalTimeWorked = daily_hours['TimeWorkedInHours'] + daily_hours['DriveTimeHours'])

# Drop col from daily_hours to not get duplicates when merged to original data later
daily_hours = daily_hours.drop(['TimeWorkedInHours','DriveTimeHours'], axis=1)


# Drop unnecessary cols in the original dataframe
df = df.drop(['ProviderFullName','Id','ClientId','ClientFirstName', 'ClientLastName','ClientContactLabels',
                   'ProviderContactLabels', 'AuthorizationId', 'AuthorizationResourceId', 'ProcedureCodeId', 'ProcedureCode', 'ProcedureCodeDescription',
                    'CodeLabels','Location','TimeWorkedInMins','ProviderFirstName','ProviderLastName',
              'UnitsOfService','DriveTimeMinutes','Mileage','BillingLabels','ClientSignature','ProviderSignature',
         'PayorId','PayorName','ProviderCharges','RateProviderDriveMileage','ProviderDriveCharge','ProviderMileageCharge',
         'ProviderChargesTotal','IsVoid','IsLocked'], axis=1)

# Merge daily_hours with ogirinal data based on ProvideId and DateOfService
merged_df = pd.merge(df, daily_hours, on=['ProviderId', 'DateOfService'])

# Get rid of rows on days that have no OT
ot_df = merged_df[merged_df['TotalTimeWorked'] > 8]


# Drop exempted providers with hourly rates = 0
ot_df = ot_df[ot_df['RateProvider'] > 0]

# Get list of provider unique ids
unique_id = ot_df['ProviderId'].unique()

# Group by ProviderId & DateOfService after merge
temp_group = ot_df.groupby(['ProviderId','DateOfService','ProviderFullName','RateProvider','RateProviderDriveHourly','TotalTimeWorked']).agg({'TimeWorkedInHours':'sum','DriveTimeHours':'sum'}).reset_index()

# Add new col for OT time of date
temp_group = temp_group.assign(OTTime = temp_group['TotalTimeWorked'] - 8)

# Add new col that finds weighted rate of the date
temp_group['OTRate'] = (temp_group['TimeWorkedInHours']*temp_group['RateProvider']+temp_group['DriveTimeHours']*temp_group['RateProviderDriveHourly'])/(temp_group['DriveTimeHours']+temp_group['TimeWorkedInHours'])

# Add empty col PartWeightedRate with partial weighted rate of the date from the work shift/row
temp_group['PartWeightedPremium'] = ""

# Add empty col for OT premium
temp_group['OTPremium'] = ""


# Loop through list of OT rows, start with going by provider
for n in unique_id:
    # For each provider, filter the temp data by the id
    filtered_by_id = temp_group[temp_group['ProviderId'] == n]
    # Get list of unique dates for the CURRENT provider
    unique_date = filtered_by_id['DateOfService'].unique()
    # For each date for the current provider
    for d in unique_date:
        # For each date, filter the temp data by the date
        filtered_by_date = filtered_by_id[filtered_by_id['DateOfService'] == d]
        # Calc the daily part of weighted rate for the entry
        filtered_by_date['PartWeightedPremium'] = ((filtered_by_date['OTRate']*(filtered_by_date['TimeWorkedInHours']+filtered_by_date['DriveTimeHours']))/filtered_by_date['TotalTimeWorked'])*filtered_by_date['OTTime']*0.5
        # Copy PartWeightedPremium from filtered_by_date to original table
        temp_group.loc[(temp_group['ProviderId'] == n) & (temp_group['DateOfService'] == d), 'PartWeightedPremium'] = filtered_by_date['PartWeightedPremium']
        # Get the daily weighted rate
        daily_premium = filtered_by_date['PartWeightedPremium'].sum()
        # Write the daily rate back to temp_group/og data        
        temp_group.loc[(temp_group['ProviderId'] == n) & (temp_group['DateOfService'] == d), 'OTPremium'] = daily_premium

# print for quick test

temp_pivot = pd.pivot_table(temp_group, values=['PartWeightedPremium'], index=['ProviderFullName','DateOfService','OTTime'], aggfunc='sum')
# print(temp_pivot)
temp_2 = temp_pivot.reset_index()
temp_2 = temp_2.groupby(['ProviderFullName']).agg({'OTTime':'sum','PartWeightedPremium':'sum'}).reset_index()

# Rename PartWeightedPremium into OTPremium to reflect sum
temp_2 = temp_2.rename(columns={'PartWeightedPremium': 'OTPremium'})

# write to new file
output_prompt = input("Enter the output dir: ")
output_filename = 'overtime_result.csv'
temp_2.to_csv(output_prompt + output_filename, mode='w', header=True)

Enter the raw file path: /Users/lymcr/Desktop/50f809494a3d46f695f1f357bf1366d9.csv


/var/folders/7y/46jvy8212rg8ctzpvvkscccw0000gn/T/ipykernel_1759/1832753302.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_by_date['PartWeightedPremium'] = ((filtered_by_date['OTRate']*(filtered_by_date['TimeWorkedInHours']+filtered_by_date['DriveTimeHours']))/filtered_by_date['TotalTimeWorked'])*filtered_by_date['OTTime']*0.5


Enter the output dir: /Users/lymcr/Desktop/
